# Microbial Genomics: Lab 6
## Topic: Burrows-Wheeler Alignments and SNP Calling
#### Tools used: BWA, SAMTools, Snippy

## Part A: Lab Exercises
### Exercise 1: Implementing a simple Burrows-Wheeler Transform
As we discussed in class, the Burrows-Wheeler algorithm is an important concept in Bioinformatics because it gives us a way to distinctly represent, align and search very large strings in an efficient way. To start this lab, we'll look a little more closely at Burrows-Wheeler Transforms.

First off, you might still be wondering about why BWT is actually important. After all, the main problem that the BWT is used to solve in Bioinformatics is searching for one block of text within a larger block. At the most basic level, we learned a few Bash commands that can search text- think of `grep`. Since sequence data is just text, can't we just use these tools to perform such searches? In theory, yes- but as with so many things in the world of Bioinformatics, the problem is a balance between memory and time.

Consider the file `sample_reads.fastq` in the lab6 folder. This file has about 75,000 sequences, or about 11,000,000 reads. Say we want to search for the sequence `GATTTCCTCCTGACCAGTCG` in this file; we could do this by executing the command below. 

*Note: we use the `time` function to get the timing statistics of the function call, and we direct the command output to `/dev/null/` as a convention to avoid a lot of noisy output in this notebook.*

In [1]:
%%bash
time cat lab6/sample_reads.fastq | grep "GATTTCCTCCTGACCAGTCG" > /dev/null


real	0m0.480s
user	0m0.470s
sys	0m0.023s


So- searching a 20-length sequence across 11M reads took about 0.5 seconds. That might not seem bad; however, when you consider that this is in fact an rather small fastq file (it's not uncommon to have bacterial sequences on the order of 100x larger), that 0.5 seconds now becomes 50 seconds. That doesn't seem so bad either, but we typically aren't just querying sequences from one organism- we are querying databases with hundreds of millions of sequences. This adds up quickly- and this is a single, relatively simple query. If we begin thinking about alignment and queries involving multiple sequences, it should be clear that in order to perform bioinformatics at scale, we need a faster solution. Indexing could speed up our search problem- but at the expense of requiring *a lot* of storage space. A single human genome would require ~200GB to index; think about trying to do this for every possible species!

So, now that we've discussed why BWT is relevant, let's talk about what it means. We discussed and practiced performing a BWT manually, but how would you do so programmatically? Let's break it down into steps, and consider the word `DOGS$` for simplicity. First, we need to generate our matrix of "rotations"; i.e., `$DOGS`, `S$DOG`, `GS$DO`, and `OGS$D`. These form our Burrows-Wheeler Matrix:

```
DOGS$
OGS$D
GS$DO
S$DOG
$DOGS
```

From a python standpoint, we could loop N times and generate these using something along the lines of the following pseudocode:

```
def rotations(word):

    rotation_list = []
    for all i from 0 to length of word:
    
        rotation_list[i] = word[i to end] + word[0 to i-1]
        
    return rotation_list
```
    
This code would return a list of all rotations of the input word. (How would you do this using a list comprehension?)

Now, we need to sort our list, so that it ends up in a form like:

```
$DOGS
DOGS$
GS$DO
OGS$D
S$DOG
```

This is equivalent to sorting the output of our `rotations` function above. Once we have this sorted BWM, we can get the BWT by taking the last letter of each row (i.e., the last letter of each element in the sorted list in Python).

Use the cell below to implement a function that takes in any arbitrary word, and returns its BWT. You can implement the internals any way you'd like, as long as the valid transformation is returned. Demonstrate that your function works on a word of your choosing after you define it.

In [10]:
# Exercise 1

def bwt(word):
    
    rotation_list = sorted([word[i:] + word[0:i] for i in range(len(word))])
    word_bwt = ''.join([x[-1] for x in rotation_list])
    return word_bwt

bwt("LOPATKIN$")

'NPKT$ILOA'

### Exercise 2: Using the `bwa` package
Above, we implemented a very simple Burrows Wheeler Transform. In reality, our Computer Science friends have been hard at work over the past 40 years making much, much more sophisticated versions of this code; there are doctoral dissertations on this topic alone. Fortunately for us, we get to reap the benefits of this hard work without getting a PhD!

There are quite a few packages that use BWT (or a related variant), but we'll focus on the `bwa`, or Burrows-Wheeler Alignment, package. You can see the basics of the tool, as always, by typing `bwa` in the command line (or the below cell):

In [2]:
%%bash
bwa

bash: line 1: bwa: command not found


CalledProcessError: Command 'b'bwa\n'' returned non-zero exit status 127.

You should note from this output that `bwa` has quite a few subcommands; the output also gives a brief description of each. The main ones of note for us are `index`, `mem`, `aln`, and `sampe`. **Do your own investigation into each of these and answer the questions below:**

1. What does each of the 4 subcommands listed above do?
2. If you were given a .fasta file and two .fastq files as inputs, and wanted to output a .sam file, what command(s) would you run?
3. Read the documentation at https://github.com/lh3/bwa. Is there more than one way to answer the above question? If so, what is the alternative to the answer you gave? Is there any reason to use one over another?

In [ ]:
# Exercise 2

### Exercise 3: Variants
Above, we learned about creating alignments with BWA; we've performed a similar workflow with Samtools in Lab 4. BWA-MEM is a much easier and faster way to perform alignments, and more importantly, brings us to our next topic: what do we do with those SAM files we generated?

One common question bioinformaticians usually want to answer when dealing with a sequence is, "how is this sequence different from a known reference"? Alignment of two sequences gives us a "birds-eye" view of the differences between them, but we can look for more granular differences in the form of single nucleotide polymorphisms (SNPs), also known as "variants". These are common mutations where a single nucleotide has changed, and can be quantified. The act of finding these variants between two genomes is known as "SNP Calling" or "Variant Calling".

To perform SNP calling, we need two things: a reference, or known, genome, and a "new" genome that we want to compare it to. Different tools have different requirements on the formats of these inputs, but there are 4 main input filetypes you will encounter:
* `.fasta`, for the reference genome: this is our well-known friend- nothing new about it.
* `.gbk`, for the reference genome: a genbank file is sometimes used as the reference input; this can be retrived from any sequence on NCBI.
* `.fastq`, for the genome to be called: some tools will accept raw fastq reads; this is the easiest option.
* `.BAM`, for genome to be called: this is a binary-compressed version of the `.SAM` format, which contains aligned, mapped sequences.

The output files will generally be one of two filetypes:
* VCF files are uncompressed files that contain variants found in a sequence compared to a reference. 
* BCF files are a binary compressed version of VCF files.

A VCF file will look something like this:
```
##fileformat=VCFv4.3
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/1000GenomesPilot-NCBI36.fasta
< ... more lines go here ... >
#CHROM POS      ID         REF   ALT    QUAL  FILTER   INFO                             FORMAT       NA00001         NA00002          NA00003
20     14370    rs6054257  G     A      29    PASS    NS=3;DP=14;AF=0.5;DB;H2           GT:GQ:DP:HQ  0|0:48:1:51,51  1|0:48:8:51,51   1/1:43:5:.,.
20     17330    .          T     A      3     q10     NS=3;DP=11;AF=0.017               GT:GQ:DP:HQ  0|0:49:3:58,50  0|1:3:5:65,3     0/0:41:3
20     1110696  rs6040355  A     G,T    67    PASS    NS=2;DP=10;AF=0.333,0.667;AA=T;DB GT:GQ:DP:HQ  1|2:21:6:23,27  2|1:2:0:18,2     2/2:35:4
20     1230237  .          T     .      47    PASS    NS=3;DP=13;AA=T                   GT:GQ:DP:HQ  0|0:54:7:56,60  0|0:48:4:51,51   0/0:61:2
20     1234567  microsat1  GTC   G,GTCT 50    PASS    NS=3;DP=9;AA=G                    GT:GQ:DP     0/1:35:4        0/2:17:2         1/1:40:3
```

Here, we have a bunch of reference information preceded with `##` at the top, followed by tab-separated variant information. Each line in the "main" table of the file represents one variant.

To try out some variant calling, we'll first use BWA to generate our aligned `.sam` files, convert to `.bam` with SAMTools, and then call variants using BCFTools, which is a sub-tool of SAMTools. The pipeline will look like this:
1. Input paired-end fastq and reference fasta files to `bwa-mem`; output a SAM file
2. Input the SAM file to `samtools view -bS` and output a BAM file
3. Sort the BAM file using `samtools sort` to output a sorted BAM file
4. Generate a BCF by using the `bcftools mpileup` and `bcftools call` commands

**Based on this information, complete the below exercises:**
1. Familiarize yourself with the SAMTools SNP-calling pipeline by reading [this guide](http://samtools.sourceforge.net/mpileup.shtml) and performing your own research on Google, Biostars, etc. Once you feel confident that you understand the steps, write a pipeline that uses reads_1.fastq, reads_2.fastq, and reference.fasta from the lab6 folder to generate a BCF file of variants between the fastq files and the reference fasta.
2. Investigate the BCF file you just generated by using the `bcftools view` command to convert your BCF file to a VCF file. Open this file in your text editor of choice. How many variants did you find?
3. Do some basic analytics of your VCF file. You can do this either by using Excel (copy all of the lines after `##` ends), or using Python, MATLAB, or your programming tool of choice. What are the most common variants you called (hint: these are the values in the `ALT` column)? 
4. Does the VCF tell us what each of these mutations actually does? If not, what other information would we need to make some judgement about the impact of each variant?

In [ ]:
# Exercise 4

### Exercise 4: SNP Calling with Snippy
SAMTools is part of the "classic" variant-calling pipeline; it was one of the very early tools developed to aid bioinformaticians in understanding their data. However, as you likely saw in the previous exercise, it is not always the most user-friendly tool. Another tool we can use to build a variant-calling pipeline is `snippy`, from the (rather prolific) Australian Bioinformatician Torsten Seemann. The github page can be found [here](https://github.com/tseemann/snippy); take some time to read through it. It is already installed in this environment, so you can view the help as usual:

In [ ]:
%%bash
snippy

Snippy is a very useful tool for several reasons:
* It can take as inputs our fastq files directly; there's no need to align them first
* It can take a Genbank file as an input, which can be directly downloaded from NCBI (i.e., using Biopython)
* It outputs a host of very useful files, most of which are very user-friendly (unlike raw VCFs)

We'll be using this tool extensively in the rest of the course, but for now, complete the exercise below using `snippy`:
1. Use Biopython Entrez to download a reference GBK file for E Coli SubSp. MG1655 (it may be useful to first do this via the [web UI](https://www.ncbi.nlm.nih.gov/nuccore/U00096.2), but for the exercise, it should be done in biopython)
2. Run `snippy` on the reads read_1.fastq and read_2.fastq in the lab6 folder; output should be placed in lab6/results/
3. Look through the output generated by `snippy`; open the .html file that was generated. Summarize and discuss your findings. 

In [ ]:
# Exercise 4

## Part B: Homework

We've talked about several different technologies in this lab; now, we'll put a few of them to use. Your homework this week will be to explore mutations in a resistant strain of *E. Coli* isolated from patients at a New York City hospital. It will be your job to write code that accomplishes the following:
1. Use BLAST to find the most closely-related strain of *E. Coli* in the NCBI database
2. Download the reference genome for that strain
3. QC and trim the input fastq files as you deem necessary
4. Align your fastq reads to the reference you downloaded, and call variants using whichever tool you prefer
5. Analyze and summarize your findings by performing a literature search, and referencing at least one paper that might describe the impacts of the mutations you found

Each step above is worth 20 points; you can write each one in a separate script, use this notebook, or write several external scripts, but you must provide a write-up of your findings in the cell below.

In [ ]:
# Homework Question 5 Writeup Goes Here